In [172]:
import numpy as np
import json
from graphviz import Digraph

In [173]:
# Layer Class
class Layer:
    def __init__(self, num_neuron: int, activation: str, weights: np.array, bias: np.array):
        self.num_neuron = num_neuron
        self.weights = weights
        self.bias = bias
        if activation == 'linear':
            self.function = lambda x: x
        elif activation == 'relu':
            self.function = lambda x: np.maximum(0, x)
        elif activation == 'sigmoid':
            self.function = lambda x: 1 / (1 + np.exp(-x))
        elif activation == 'softmax':
            self.function = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)
        else:
            raise ValueError('Invalid activation function')

    def forward(self, input: np.array):
        return self.function(np.dot(input, self.weights) + self.bias)

In [174]:
# FFNN Class
class FFNN:
    def __init__(self, input_size:int, input: np.array, layers: list):
        self.input_size = input_size
        self.input = input
        self.layers = layers
        self.output = None

    def add_layer(self, layer: Layer):
        self.layers.append(layer)

    def forward(self):
        self.output = self.input
        for layer in self.layers:
            self.output = layer.forward(self.output)
            print(layer.weights)
        return self.output

    def visualize(self):
        dot = Digraph(comment="Feed Forward Neural Network")

        # if theres only input and output layer
        if len(self.layers) == 1:
            for i in range(self.input_size):
                for j in range(len(self.output[0])):
                    dot.edge(f"input{i+1}", f"output{j+1}", label=f"{self.layers[0].weights[i][j]}")
        
        # TODO: perfect the visualization
        # if theres hidden layer
        else:
            # Input Layer
            for i in range(self.input_size):
                dot.node(f"Input {i+1}", f"Input {i+1}")
            
            # Hidden Layer
            for i in range(len(self.layers)):
                for j in range(self.layers[i].num_neuron):
                    dot.node(f"Hidden {i+1}.{j+1}", f"Hidden {i+1}.{j+1}")

                if i == 0:
                    for k in range(self.input_size):
                        for j in range(self.layers[i].num_neuron):
                            dot.edge(f"Input {k+1}", f"Hidden {i+1}.{j+1}", label=f"{self.layers[i].weights[k][j]}")
                
                else:
                    for k in range(self.layers[i-1].num_neuron):
                        for j in range(self.layers[i].num_neuron):
                            dot.edge(f"Hidden {i}.{k+1}", f"Hidden {i+1}.{j+1}", label=f"{self.layers[i].weights[k][j]}")
            
            # Output Layer
            for i in range(len(self.output[0])):
                dot.node(f"Output {i+1}", f"Output {i+1}")
                print(f"Output {i+1}")
            
            for i in range(self.layers[-1].num_neuron):
                for j in range(len(self.output[0])):
                    dot.edge(f"Hidden {len(self.layers)}.{i+1}", f"Output {j+1}", label=f"{self.output[i][j]}")
        
        return dot

In [175]:
input_file = str(input("Enter the input file name (JSON only): "))
# input_file = "multilayer.json"
with open(f"test-case/{input_file}.json", "r") as file:
    model = json.load(file)

input_size = model["case"]["model"]["input_size"]
input_array = np.array(model["case"]["input"])
layers = model["case"]["model"]["layers"]
ffnn = FFNN(input_size, input_array, [])
for i in range(len(layers)):
    layer = layers[i]
    weight = model["case"]["weights"][i]
    new_layer = Layer(layer["number_of_neurons"], layer["activation_function"], np.array(weight[1:]), np.array(weight[0]))
    ffnn.add_layer(new_layer)

ffnn.forward()
print(ffnn.output)

[[-0.5  0.6  0.7  0.5]
 [ 0.9  1.  -1.1 -1. ]
 [ 1.3  1.4  1.5  0.1]]
[[-0.4  0.5  0.6]
 [ 0.7  0.4 -0.9]
 [ 0.2  0.3  0.4]
 [-0.1  0.2  0.1]]
[[-0.3  0.4]
 [ 0.6  0.1]
 [ 0.1 -0.4]]
[[-0.2]
 [ 0.3]]
[[0.4846748]]


In [176]:
dot = ffnn.visualize()
dot.render("model_visual", format="png", cleanup=True)

Output 1


'model_visual.png'